In [1]:
# Give an overview about CARD 
## downlod database
## create blast databases
## create blast 
## run blast database
## check quality of ARG
## report good quality ARG. 

In [ ]:
## to download the most updated version of resfinder
mkdir $ASM/Databases/Resfinder
cd $ASM/Databases/Resfinder
git clone https://bitbucket.org/genomicepidemiology/resfinder_db.git
cd resfinder_db


## make sure all files have a linux format

ls *fsa | while read file; do dos2unix $file; done 


## compile them all in one file.

cat *.fsa > ../All.resfinder.fsa



##ls *fsa | while read file; do cat $file | awk -F"\t" '{print } END {print "\n"}'; done  > ../All.resfinder.fsa

## OPTIONAL
 ls *fsa | while read file ; do f=$(echo "$file" | cut -f1 -d.); grep ">" $file | 
 sed 's/>//g' | awk -F"\t" -v class="$f" '{print $1"\t"class}'; done | 
 awk -F"\t" 'BEGIN {print "Gene\tClass"} {print $1"\t"$2 }'  > ../Resfinder.gene.class

cd ..


## OPTIONAL
## translate the NA resfinder to protein
$EmbossFolder/transeq -sequence All.resfinder.fsa -outseq All.resfinder.AA.fa -frame 1

head All.resfinder.prot.fa
sed 's/_[1-6]$//g' All.resfinder.AA.fa  | head 

sed 's/_[1-6]$//g' All.resfinder.AA.fa -i

mkdir blastNA  ## blastAA

## OPTIONAL
## compiling blast Protein database for Resfinder
cd blastAA
$blastExecFolder/makeblastdb -in ../All.resfinder.AA.fa -title RESFINDERProt \
-out RESFINDERProt -input_type fasta   -hash_index -dbtype prot

echo "export RESFINDERProt=$(pwd)/RESFINDERProt" >> ~/.bashrc
tail -n 1 ~/.bashrc
source ~/.bashrc

cd ..
## compiling blast Nucleotides database for Resfinder


## install binary blast 

cd blastNA

$blastExecFolder/makeblastdb -in ../All.resfinder.fsa -title RESFINDERNucl \
-out RESFINDERNucl -input_type fasta   -hash_index -dbtype nucl

## REMOVE IT
echo "export RESFINDERNucl=$(pwd)/RESFINDERNucl" >> ~/.bashrc
tail -n 1 ~/.bashrc
source ~/.bashrc




In [ ]:
## Run CARD databses

cd $ASM/Data
mkdir Resfinder
cd Resfinder
## nucleotide versus Resfinder nucleotide

## replace  RESFINDERNucl with right path.
time $blastExecFolder/blastn -query ../orfs.nucleotide.fa -db $RESFINDERNucl -outfmt 6 \
 -max_target_seqs 1 -evalue 1E-50 -word_size 6 -num_threads 1 -out orf.resfinder.NA.versus.NA.tab 
 
 
## protein versus Resfinder nucleotide
$blastExecFolder/tblastn -query ../orfs.protein.fa -db $RESFINDERNucl -outfmt 6 \
 -max_target_seqs 10 -evalue 1E-50 -word_size 3 -num_threads 28 -out orf.resfinder.AA.versus.NA.tab 
 
## protein versus Resfinder nucleotide
$blastExecFolder/tblastx -query ../orfs.nucleotide.fa -db $RESFINDERNucl -outfmt 6 \
 -max_target_seqs 10 -evalue 1E-50 -word_size 3 -num_threads 28 -out orf.resfinder.trans.NA.versus.NA.tab 
 
## protein versus Resfinder protein
$blastExecFolder/blastp -query  ../orfs.protein.fa -db $RESFINDERProt -outfmt 6 \
 -max_target_seqs 10 -evalue 1E-50 -word_size 6 -num_threads 28  -out orf.resfinder.AA.versus.AA.tab 
 
## nucleotide versus Resfinder protein
$blastExecFolder/blastx -query ../orfs.nucleotide.fa -db $RESFINDERProt -outfmt 6 \
 -max_target_seqs 10 -evalue 1E-50 -word_size 6 -num_threads 28  -out orf.resfinder.NA.versus.AA.tab 




In [ ]:
## fruther annotation
## maping gene to class

## 
Resfinder.gene.class 
JoinTwoFilesBasedOnKeys.sh 2 1 $ASM/Databases/Resfinder/Resfinder.gene.class orf.resfinder.NA.versus.NA.tab 

## how many gene classes are there?
## how many betalactamases are there?
## how many inserts have resistance gene?
